##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [1]:
!pip install -U -q "google-generativeai>=0.8.2"

In [12]:
ls

2.jpg  sample_data/  tempfiles/


In [18]:
import os
import google.generativeai as genai

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

def upload_to_gemini(path, mime_type=None):
  """Uploads the given file to Gemini.

  See https://ai.google.dev/gemini-api/docs/prompting_with_media
  """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file

# Create the model
generation_config = {
  "temperature": 0,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-pro",
  generation_config=generation_config,
)

# TODO Make these files available on the local file system
# You may need to update the file paths
files = [
  upload_to_gemini("2.jpg", mime_type="image/jpeg"),
  upload_to_gemini("1.jpg", mime_type="image/jpeg"),
]

chat_session = model.start_chat(
  history=[{
      "role": "user",
      "parts": [
        files[0],
        "该图是一个需要进行区域供热系统规划设计的对象，现有热源和六个负荷，需要你进行负责设计一个供热系统拓扑结构。所构建的拓扑结构需要严格遵循以下：你需要对各个对象和管道交接处设置节点，节点连接代表管道，这些管道不能穿过建筑只在图中的道路进行设计。\n如果同向有多个负荷，那么该主管道需要进行分支，并且分支后的新节点连接至这两个负荷。\n同一个节点不能连接超过3个管道，如果有需要设计管道分支。\n\n参考：热源节点编号为1；总部基地节点编号为2；商务中心节点编号为3；绿创大厦节点编号为4；北郡节点编号为5；光明府节点编号为6；壹号天禧节点编号为7。你还需要对其他节点额外定义编号。\n\n请你用返回一个列表，其中列表元素包含（node_id1,node_id2）代表从节点编号node_id1流入节点编号node_id2。\n结果模版例如：“[\n(1, 5), # 热源 -> 节点5\n(5, 6), # 节点5 -> 节点6\n(6, 2), # 节点6 -> 总部基地\n(6, 4), # 节点6 -> 壹号天禧\n(5, 3) # 节点5 -> 商务中心\n]”",
      ],
    },
  {
      "role": "model",
      "parts": [
        "```\n[\n(1, 8),  # 热源 -> 节点8\n(8, 9),  # 节点8 -> 节点9\n(9, 2),  # 节点9 -> 总部基地\n(9, 7),  # 节点9 -> 壹号天禧\n(8, 10), # 节点8 -> 节点10\n(10, 11),# 节点10 -> 节点11\n(11, 5), # 节点11 -> 北郡\n(11, 6), # 节点11 -> 光明府\n(10, 3)  # 节点10 -> 商务中心\n]\n```",
      ],
    },
  ]
)

response = chat_session.send_message("INSERT_INPUT_HERE")

print(response.text)

Uploaded file '2.jpg' as: https://generativelanguage.googleapis.com/v1beta/files/zaj55mn7tybp
```
[
(1, 8), # 热源 -> 节点8
(8, 9), # 节点8 -> 节点9
(9, 2), # 节点9 -> 总部基地
(9, 7),  # 节点9 -> 壹号天禧
(8, 10), # 节点8 -> 节点10
(10, 11), # 节点10 -> 节点11
(11, 4), # 节点11 -> 绿创大厦
(11, 12), # 节点11 -> 节点12
(12, 3), # 节点12 -> 商务中心
(10, 13), # 节点10 -> 节点13
(13, 5), # 节点13 -> 北郡
(13, 6) # 节点13 -> 光明府
]
```


In [ ]:
{
      "role": "user",
      "parts": [
        files[1],
        prompt],
    },{
      "role": "model",
      "parts": ["""[
(1, 4), # 热源 -> 节点4
(4, 2), # 节点4 -> load1
(4, 3) # 节点4 -> load2
]"""],
    },

In [37]:
prompt="""该图是一个需要进行区域供热系统规划设计的对象，现有热源和六个负荷，需要你进行负责设计一个供热系统拓扑结构。
所构建的拓扑结构需要严格遵循以下规则：
1.你需要将热源和各个负荷通过节点进行连接。节点连接代表管道，但是，这些管道不能穿过建筑只在图中的道路进行设计，你需要不断反思设计的结构是否会穿过图中的建筑或者湖。
2.你必须先理解图中各个负荷之间以及热源的相对位置关系。如果该节点同向有两个负荷，那么需要新增支路分支节点，再分别连接至这两个负荷。
3.每个新增的支路节点（除了负荷节点和热源节点）必须连接其他3个节点，如果有需要设计管道分支。
4.热源节点只能作为源节点，负荷节点只能作为目标节点。

参考：热源节点编号为1；总部基地节点编号为2；商务中心节点编号为3；绿创大厦节点编号为4；北郡节点编号为5；光明府节点编号为6；壹号天禧节点编号为7。你还需要对其他新增的支路节点编号额外进行定义。

请你用返回一个列表，其中列表元素包含（源节点node_id1,目标节点node_id2），代表从源节点编号node_id1流入目标节点编号node_id2。
结果模版例如：“[
(1, 5), # 热源 -> 节点5
(5, 6), # 节点5 -> 节点6
(6, 2), # 节点6 -> 总部基地
(6, 4), # 节点6 -> 壹号天禧
(5, 3) # 节点5 -> 商务中心
]”"""

In [38]:
history_ = [
    {
      "role": "user",
      "parts": [
        files[1],
        prompt],
    },
    {
      "role": "model",
      "parts": ["""[
(1, 4), # 热源 -> 节点4
(4, 2), # 节点4 -> load1
(4, 3) # 节点4 -> load2
]"""],
    },
    {
      "role": "user",
      "parts": [
        files[0],
        prompt],
    },
  ]
for i in range(5):
  chat_session = model.start_chat(
  history=history_
  )
  response = chat_session.send_message("INSERT_INPUT_HERE")
  print(response.text)
  model_results = {
      "role": "model",
      "parts": [response.text],
    }
  history_.append(model_results)

  model_start = {
      "role": "user",
      "parts": [
        "请你思考这个结果是否合理，是否遵循规则，如果遵守则输出mark=1，如果不遵守根据规则改进，重新设计拓扑结构。"
        ],
    }
  history_.append(model_start)

```python
[
(1, 8),  # 热源 -> 节点8
(8, 2),  # 节点8 -> 总部基地
(8, 9),  # 节点8 -> 节点9
(9, 3),  # 节点9 -> 商务中心
(9, 7),  # 节点9 -> 壹号天禧
(9, 10), # 节点9 -> 节点10
(10, 6), # 节点10 -> 光明府
(10, 5), # 节点10 -> 北郡
(10, 4)  # 节点10 -> 绿创大厦
]
```

你的拓扑结构不完全符合规则3：每个新增的支路节点（除了负荷节点和热源节点）必须连接其他3个节点。

你的方案中：
* 节点8连接了3个节点（热源，总部基地，节点9） - 符合规则
* 节点9连接了4个节点（节点8，商务中心，壹号天禧，节点10）- **不符合规则**
* 节点10连接了4个节点（节点9，光明府，北郡，绿创大厦）- **不符合规则**

为了满足规则3，我们需要调整拓扑结构，减少节点9和节点10的连接数。一个改进方案如下：

```
[
(1, 8),  # 热源 -> 节点8
(8, 2),  # 节点8 -> 总部基地
(8, 11), # 节点8 -> 节点11
(11, 3), # 节点11 -> 商务中心
(11, 9), # 节点11 -> 节点9
(9, 7),  # 节点9 -> 壹号天禧
(9, 10), # 节点9 -> 节点10
(10, 6), # 节点10 -> 光明府
(10, 12),# 节点10 -> 节点12
(12, 5), # 节点12 -> 北郡
(12, 4)  # 节点12 -> 绿创大厦
]
```

在这个改进方案中：

* 节点8连接3个节点 (热源, 总部基地, 节点11) - 符合规则
* 节点11连接3个节点 (节点8, 商务中心, 节点9) - 符合规则
* 节点9连接3个节点 (节点11, 壹号天禧, 节点10) - 符合规则
* 节点10连接3个节点 (节点9, 光明府, 节点12) - 符合规则
* 节点12连接3个节点 (节点10, 北郡, 绿创大厦) - 符合规则

所有新增节点都连接了3个其他节点，符合所有规则。因此，mark = 1


最终答案：

```
[
(1, 8),  # 热源 -> 节点8
(8, 2),  # 节点8 -> 总部

TooManyRequests: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).

In [4]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-1.5-pro' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3siZmlsZV9kYXRhIjp7Im1pbWVfdHlwZSI6ImFwcGxpY2F0aW9uL29jdGV0LXN0cmVhbSIsImRyaXZlX2lkIjoiMXZlQU9xTkVLendBZG4wN1pmcHJUbEVRcFJKRGl6U21DIn19LHsidGV4dCI6IuivpeWbvuaYr+S4gOS4qumcgOimgei/m+ihjOWMuuWfn+S+m+eDreezu+e7n+inhOWIkuiuvuiuoeeahOWvueixoe+8jOeOsOacieeDrea6kOWSjOWFreS4qui0n+iNt++8jOmcgOimgeS9oOi/m+ihjOi0n+i0o+iuvuiuoeS4gOS4quS+m+eDreezu+e7n+aLk+aJkee7k+aehOOAguS9oOmcgOimgeWvueWQhOS4quWvueixoeWSjOeuoemBk+S6pOaOpeWkhOiuvue9ruiKgueCue+8jOiKgueCuei/nuaOpeS7o+ihqOeuoemBk++8jOi/meS6m+euoemBk+S4jeiDveepv+i/h+W7uuetkeWPquWcqOWbvuS4reeahOmBk+i3r+i/m+ihjOiuvuiuoeOAglxcbuWmguaenOWQjOWQkeacieWkmuS4qui0n+iNt++8jOmCo+S5iOivpeS4u+euoemBk+mcgOimgei/m+ihjOWIhuaUr++8jOW5tuS4lOWIhuaUr+WQjueahOaWsOiKgueCuei/nuaOpeiHs+i/meS4pOS4qui0n+iNt+OAglxcbuWQjOS4gOS4quiKgueCueS4jeiDvei/nuaOpei2hei/hzPkuKrnrqHpgZPvvIzlpoLmnpzmnInpnIDopoHorr7orqHnrqHpgZPliIbmlK/jgIJcXG5cXG7lj4LogIPvvJrng63mupDoioLngrnnvJblj7fkuLox77yb5oC76YOo5Z+65Zyw6IqC54K557yW5Y+35Li6Mu+8m+WVhuWKoeS4reW/g+iKgueCuee8luWPt+S4ujPvvJvnu7/liJvlpKfljqboioLngrnnvJblj7fkuLo077yb5YyX6YOh6IqC54K557yW5Y+35Li6Ne+8m+WFieaYjuW6nOiKgueCuee8luWPt+S4ujbvvJvlo7nlj7flpKnnpqfoioLngrnnvJblj7fkuLo344CC5L2g6L+Y6ZyA6KaB5a+55YW25LuW6IqC54K56aKd5aSW5a6a5LmJ57yW5Y+344CCXFxuXFxu6K+35L2g55So6L+U5Zue5LiA5Liq5YiX6KGo77yM5YW25Lit5YiX6KGo5YWD57Sg5YyF5ZCr77yIbm9kZV9pZDEsbm9kZV9pZDLvvInku6Pooajku47oioLngrnnvJblj7dub2RlX2lkMea1geWFpeiKgueCuee8luWPt25vZGVfaWQy44CCXFxu57uT5p6c5qih54mI5L6L5aaC77ya4oCcW1xcbigxLCA1KSwgIyDng63mupAgLT4g6IqC54K5NVxcbig1LCA2KSwgIyDoioLngrk1IC0+IOiKgueCuTZcXG4oNiwgMiksICMg6IqC54K5NiAtPiDmgLvpg6jln7rlnLBcXG4oNiwgNCksICMg6IqC54K5NiAtPiDlo7nlj7flpKnnpqdcXG4oNSwgMykgIyDoioLngrk1IC0+IOWVhuWKoeS4reW/g1xcbl3igJ0ifV19LHsicm9sZSI6Im1vZGVsIiwicGFydHMiOlt7InRleHQiOiJgYGBcXG5bXFxuKDEsIDgpLCAgIyDng63mupAgLT4g6IqC54K5OFxcbig4LCA5KSwgICMg6IqC54K5OCAtPiDoioLngrk5XFxuKDksIDcpLCAgIyDoioLngrk5IC0+IOWjueWPt+Wkqeemp1xcbig5LCAyKSwgICMg6IqC54K5OSAtPiDmgLvpg6jln7rlnLBcXG4oOCwgMTApLCAjIOiKgueCuTggLT4g6IqC54K5MTBcXG4oMTAsIDExKSwjIOiKgueCuTEwIC0+IOiKgueCuTExXFxuKDExLCA0KSwgIyDoioLngrkxMSAtPiDnu7/liJvlpKfljqZcXG4oMTEsIDMpLCAjIOiKgueCuTExIC0+IOWVhuWKoeS4reW/g1xcbigxMCwgMTIpLCMg6IqC54K5MTAgLT4g6IqC54K5MTJcXG4oMTIsIDUpLCAjIOiKgueCuTEyIC0+IOWMl+mDoVxcbigxMiwgNiksICMg6IqC54K5MTIgLT4g5YWJ5piO5bqcXFxuXVxcbmBgYCJ9XX1d' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MCwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):

        # if drive is None:
        from google.colab import drive
        drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

Mounted at /gdrive
Uploading: /gdrive/.shortcut-targets-by-id/1veAOqNEKzwAdn07ZfprTlEQpRJDizSmC/2.jpg
[
    {
        "role": "user",
        "parts": [
            {
                "file_data": {
                    "mime_type": "application/octet-stream",
                    "file_uri": "https://generativelanguage.googleapis.com/v1beta/files/n9dvci9kknis"
                }
            },
            {
                "text": "\u8be5\u56fe\u662f\u4e00\u4e2a\u9700\u8981\u8fdb\u884c\u533a\u57df\u4f9b\u70ed\u7cfb\u7edf\u89c4\u5212\u8bbe\u8ba1\u7684\u5bf9\u8c61\uff0c\u73b0\u6709\u70ed\u6e90\u548c\u516d\u4e2a\u8d1f\u8377\uff0c\u9700\u8981\u4f60\u8fdb\u884c\u8d1f\u8d23\u8bbe\u8ba1\u4e00\u4e2a\u4f9b\u70ed\u7cfb\u7edf\u62d3\u6251\u7ed3\u6784\u3002\u4f60\u9700\u8981\u5bf9\u5404\u4e2a\u5bf9\u8c61\u548c\u7ba1\u9053\u4ea4\u63a5\u5904\u8bbe\u7f6e\u8282\u70b9\uff0c\u8282\u70b9\u8fde\u63a5\u4ee3\u8868\u7ba1\u9053\uff0c\u8fd9\u4e9b\u7ba1\u9053\u4e0d\u80fd\u7a7f\u8fc7\u5efa\u7b51\u53ea\u5728\u56fe\u4

## Call `generate_content`

In [9]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream
)

display(Markdown(response.text))

InvalidArgument: 400 Unable to submit request because it has a mimeType parameter with value application/octet-stream, which is not supported. Update the mimeType and try again. Learn more: https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/gemini

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [ ]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")